<a href="https://colab.research.google.com/github/alaa7amid/AI/blob/main/clusteringEstimate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
class DataCloud:
	N=0
	def __init__(self,x):
		self.n=1
		self.mean=x
		self.variance=0
		self.pertinency=1
		DataCloud.N+=1
	def addDataClaud(self,x):
		self.n=2
		self.mean=(self.mean+x)/2
		self.variance=((np.linalg.norm(self.mean-x))**2)
	def updateDataCloud(self,n,mean,variance):
		self.n=n
		self.mean=mean
		self.variance=variance

In [ ]:
class AutoCloud:
	c= np.array([DataCloud(0)],dtype=DataCloud)
	alfa= np.array([0.0],dtype=float)
	intersection = np.zeros((1,1),dtype=int)
	listIntersection = np.zeros((1),dtype=int)
	matrixIntersection = np.zeros((1,1),dtype=int)
	relevanceList = np.zeros((1),dtype=int)
	k=1
	def __init__(self, m):
		AutoCloud.m = m
		AutoCloud.c = np.array([DataCloud(0)],dtype=DataCloud)
		AutoCloud.alfa= np.array([0.0],dtype=float)
		AutoCloud.intersection = np.zeros((1,1),dtype=int)
		AutoCloud.listIntersection = np.zeros((1),dtype=int)
		AutoCloud.relevanceList = np.zeros((1),dtype=int)
		AutoCloud.matrixIntersection = np.zeros((1,1),dtype=int)
		AutoCloud.k=1
		AutoCloud.classIndex = [[1.0],[1.0]]
		AutoCloud.clusterIndex = [0, 0]

	def mergeClouds(self):
		i=0
		while(i<len(AutoCloud.listIntersection)-1):
			merge=False
			j=i+1
			while(j<len(AutoCloud.listIntersection)):
				#print("i",i,"j",j,"l",np.size(AutoCloud.listIntersection),"m",np.size(AutoCloud.matrixIntersection),"c",np.size(AutoCloud.c))
				if(AutoCloud.listIntersection[i] == 1 and AutoCloud.listIntersection[j] == 1):
					AutoCloud.matrixIntersection[i,j] = AutoCloud.matrixIntersection[i,j] + 1;
				nI = AutoCloud.c[i].n
				nJ = AutoCloud.c[j].n
				meanI = AutoCloud.c[i].mean
				meanJ = AutoCloud.c[j].mean
				varianceI = AutoCloud.c[i].variance
				varianceJ = AutoCloud.c[j].variance
				nIntersc = AutoCloud.matrixIntersection[i,j]
				if (nIntersc > (nI - nIntersc) or nIntersc > (nJ - nIntersc)):
					merge = True
					#update values
					n = nI + nJ - nIntersc
					mean = ((nI * meanI) + (nJ * meanJ))/(nI + nJ)
					variance = ((nI - 1) * varianceI + (nJ - 1) * varianceJ)/(nI + nJ - 2)
					newCloud = DataCloud(mean)
					newCloud.updateDataCloud(n,mean,variance)
					#atualizando lista de interseção
					AutoCloud.listIntersection = np.concatenate((AutoCloud.listIntersection[0 : i], np.array([1]), AutoCloud.listIntersection[i + 1 : j],AutoCloud.listIntersection[j + 1 : np.size(AutoCloud.listIntersection)]),axis=None)
					#atualizando lista de data clouds
					AutoCloud.c = np.concatenate((AutoCloud.c[0 : i ], np.array([newCloud]), AutoCloud.c[i + 1 : j],AutoCloud.c[j + 1 : np.size(AutoCloud.c)]),axis=None)
					#update  intersection matrix
					M0 = AutoCloud.matrixIntersection
					#Remover linhas
					M1=np.concatenate((M0[0 : i , :],np.zeros((1,len(M0))),M0[i + 1 : j, :],M0[j + 1 : len(M0), :]))
					#remover colunas
					M1=np.concatenate((M1[:, 0 : i ],np.zeros((len(M1),1)),M1[:, i+1 : j],M1[:, j+1 : len(M0)]),axis=1)
					#calculando nova coluna
					col = (M0[:, i] + M0[:, j])*(M0[: , i]*M0[:, j] != 0)
					col = np.concatenate((col[0 : j], col[j + 1 : np.size(col)]))
					#calculando nova linha
					lin = (M0[i, :]+M0[j, :])*(M0[i, :]*M0[j, :] != 0)
					lin = np.concatenate((lin[ 0 : j], lin[j + 1 : np.size(lin)]))
					#atualizando coluna
					M1[:,i]=col
					#atualizando linha
					M1[i,:]=lin
					M1[i, i + 1 : j] = M0[i, i + 1 : j] + M0[i + 1 : j, j].T;
					AutoCloud.matrixIntersection = M1
				j += 1
			if(merge):
				i = 0
			else:
				i += 1

	def run(self,X):
		AutoCloud.listIntersection = np.zeros((np.size(AutoCloud.c)),dtype=int)
		if AutoCloud.k==1:
			AutoCloud.c[0]=DataCloud(X)

		elif AutoCloud.k==2:
			AutoCloud.c[0].addDataClaud(X)
		elif AutoCloud.k>=3:
			i=0
			createCloud = True
			AutoCloud.alfa = np.zeros((np.size(AutoCloud.c)),dtype=float)
			for data in AutoCloud.c:
				n= data.n +1
				mean = ((n-1)/n)*data.mean + (1/n)*X
				variance = ((n-1)/n)*data.variance +(1/n)*((np.linalg.norm(X-mean))**2)
				eccentricity=(1/n)+((mean-X).T.dot(mean-X))/(n*variance)
				typicality = 1 - eccentricity
				norm_eccentricity = eccentricity/2
				norm_typicality = typicality/(AutoCloud.k-2)
				if(norm_eccentricity<=(AutoCloud.m**2 +1)/(2*n)):
					data.updateDataCloud(n,mean,variance)
					AutoCloud.alfa[i] = norm_typicality
					createCloud= False
					AutoCloud.listIntersection.itemset(i,1)
				else:
					AutoCloud.alfa[i] = norm_typicality
					AutoCloud.listIntersection.itemset(i,0)
				i+=1

			if(createCloud):
				AutoCloud.c = np.append(AutoCloud.c,DataCloud(X))
				AutoCloud.listIntersection = np.insert(AutoCloud.listIntersection,i,1)
				AutoCloud.matrixIntersection = np.pad(AutoCloud.matrixIntersection, ((0,1),(0,1)), 'constant', constant_values=(0))
			self.mergeClouds()
			AutoCloud.relevanceList = AutoCloud.alfa /np.sum(AutoCloud.alfa)
			AutoCloud.clusterIndex.append(np.argmax(AutoCloud.relevanceList))
			AutoCloud.classIndex.append(AutoCloud.alfa)


		AutoCloud.k=AutoCloud.k+1

In [ ]:
def Dmin(c):
	dmin = np.linalg.norm(np.subtract(c[0].mean,c[1].mean))
	for i in range(len(c)):
		for j in range(len(c)):
			if(i != j):
				if(np.linalg.norm(np.subtract(c[i].mean,c[j].mean))<dmin):
					dmin = np.linalg.norm(np.subtract(c[i].mean,c[j].mean))
	return dmin

## 1. Spiral

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/AutoCloud-master (5)/Spiral.csv')
data = data.dropna()
#dados = np.array([data["1"],data["2"]])
#dados= dados.T
#class_labels = np.array(data["3"])

In [ ]:

teste=AutoCloud(2)
purity=[]
accuracy=[]
NMI=[]
for t in dados:
    teste.run(np.array(t))
    if teste.k>2:
        max_idx=max(teste.clusterIndex)
        indexed_data=pd.DataFrame(class_labels[0:teste.k-1],columns=["classLabel"])
        indexed_data["clusterLabel"]=teste.clusterIndex
        sorted_data=indexed_data.value_counts().to_frame()
        C=sorted_data.groupby(["clusterLabel"]).sum() # clustered number of data
        n_d=[]
        precision=[]
        HY_c=[]
        for idx in range(max_idx+1):
            if (1,idx) not in sorted_data.index:
                tmp1=0
                tmp2=sorted_data.loc[(2,idx),0]
                p_c2=tmp2/(tmp1+tmp2)
                HY_c_tmp=-C.loc[idx,0]/C.sum()[0]*p_c2*np.log2(p_c2)
            elif (2,idx) not in sorted_data.index:
                tmp1=sorted_data.loc[(1,idx),0]
                tmp2=0
                p_c1=tmp1/(tmp1+tmp2)
                HY_c_tmp=-C.loc[idx,0]/C.sum()[0]*p_c1*np.log2(p_c1)
            else:
                tmp1=sorted_data.loc[(1,idx),0]
                tmp2=sorted_data.loc[(2,idx),0]
                p_c1=tmp1/(tmp1+tmp2)
                p_c2=tmp2/(tmp1+tmp2)
                HY_c_tmp=-C.loc[idx,0]/C.sum()[0]*(p_c1*np.log2(p_c1)+p_c2*np.log2(p_c2))
            n_d.append(max(tmp1,tmp2))
            precision.append(max(tmp1,tmp2)/C.loc[idx,0])
            HY_c.append(HY_c_tmp)
        purity.append(sum(n_d)/teste.k)
        accuracy.append(np.mean(precision))

        # entropy of class labels
        Y=sorted_data.groupby(["classLabel"]).sum()
        p=Y/np.sum(Y)
        HY=-np.sum(p*np.log2(p))[0]
        # entropy of cluster labels
        p=C/np.sum(C)
        HC=-np.sum(p*np.log2(p))[0]
        # mutual information
        I=HY-sum(HY_c)
        # Thus, the NMI
        NMI.append(2*I/(HY+HC))


In [ ]:
sampleSpeed=10 # samples per time unit
t=np.linspace(0,len(purity)/sampleSpeed,len(purity))

In [ ]:
sample_speed=[5,10,15,20,25]
avgPurity=[]
for speed in sample_speed:
    tmp=[]
    for i in range(1,len(purity)//speed):
        tmp.append(purity[speed*i])
    avgPurity.append(sum(tmp)/len(tmp)*100) # [%]

# creating the bar plot
plt.bar(sample_speed, avgPurity, color = 'blue', width = 3)
plt.xlabel("Sample Speed (PPS)")
plt.ylabel("Avg Purity (%)")
plt.show()

In [ ]:
# purity
plt.plot(t,purity)
plt.ylim([0,1])
plt.xlabel('Time [unit]')
plt.ylabel("Purity")
plt.show()

In [ ]:
print("Purity:")
print(purity)

In [ ]:
# accuracy
plt.plot(t,accuracy)
plt.ylim([0,1])
plt.xlabel('Time [unit]')
plt.ylabel("Accuracy")
plt.show(purity)

In [ ]:
print("Accuracy:")
print(accuracy)

In [ ]:
# NMI
plt.plot(t,NMI)
plt.xlabel('Time [unit]')
plt.ylabel("NMI")
plt.show()

In [ ]:
print("NMI:")
print(NMI)

In [ ]:
plt.rcParams["figure.figsize"] = (18,12)
plt.grid()
dados= dados.T
plt.plot(dados[0],dados[1],'.g')
for i in range(0,np.size(teste.c)):
    plt.plot(teste.c[i].mean[0],teste.c[i].mean[1],'x',color='black')
plt.legend(['Samples','Auto-Cloud'])
plt.show()

## 2. KDDCUP99

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/AutoCloud-master (5)/KDDCUP99.csv', header=None)
data = data.dropna()
dados = np.array(data.loc[:,0:41])
class_labels = np.array(data.loc[:,42])

In [ ]:
teste=AutoCloud(42)
purity=[]
accuracy=[]
NMI=[]
for t in dados:
    teste.run(np.array(t))
    if teste.k>2:
        max_idx=max(teste.clusterIndex)
        indexed_data=pd.DataFrame(class_labels[0:teste.k-1],columns=["classLabel"])
        indexed_data["clusterLabel"]=teste.clusterIndex
        sorted_data=indexed_data.value_counts().to_frame()
        C=sorted_data.groupby(["clusterLabel"]).sum() # clustered number of data
        n_d=[]
        precision=[]
        HY_c=[]
        for idx in range(max_idx+1):
            if (1,idx) not in sorted_data.index:
                tmp1=0
                tmp2=sorted_data.loc[(2,idx),0]
                p_c2=tmp2/(tmp1+tmp2)
                HY_c_tmp=-C.loc[idx,0]/C.sum()[0]*p_c2*np.log2(p_c2)
            elif (2,idx) not in sorted_data.index:
                tmp1=sorted_data.loc[(1,idx),0]
                tmp2=0
                p_c1=tmp1/(tmp1+tmp2)
                HY_c_tmp=-C.loc[idx,0]/C.sum()[0]*p_c1*np.log2(p_c1)
            else:
                tmp1=sorted_data.loc[(1,idx),0]
                tmp2=sorted_data.loc[(2,idx),0]
                p_c1=tmp1/(tmp1+tmp2)
                p_c2=tmp2/(tmp1+tmp2)
                HY_c_tmp=-C.loc[idx,0]/C.sum()[0]*(p_c1*np.log2(p_c1)+p_c2*np.log2(p_c2))
            n_d.append(max(tmp1,tmp2))
            precision.append(max(tmp1,tmp2)/C.loc[idx,0])
            HY_c.append(HY_c_tmp)
        purity.append(sum(n_d)/teste.k)
        accuracy.append(np.mean(precision))

        # entropy of class labels
        Y=sorted_data.groupby(["classLabel"]).sum()
        p=Y/np.sum(Y)
        HY=-np.sum(p*np.log2(p))[0]
        # entropy of cluster labels
        p=C/np.sum(C)
        HC=-np.sum(p*np.log2(p))[0]
        # mutual information
        I=HY-sum(HY_c)
        # Thus, the NMI
        NMI.append(2*I/(HY+HC))



In [ ]:
sampleSpeed=10 # samples per time unit
t=np.linspace(0,len(purity)/sampleSpeed,len(purity))

In [ ]:
sample_speed=[5,10,15,20,25]
avgPurity=[]
for speed in sample_speed:
    tmp=[]
    for i in range(1,len(purity)//speed):
        tmp.append(purity[speed*i])
    avgPurity.append(sum(tmp)/len(tmp)*100) # [%]

# creating the bar plot
plt.bar(sample_speed, avgPurity, color = 'blue', width = 3)
plt.xlabel("Sample Speed (PPS)")
plt.ylabel("Avg Purity (%)")
plt.show()

In [ ]:
# purity
plt.plot(t,purity)
plt.ylim([0,1])
plt.xlabel('Time [unit]')
plt.ylabel("Purity")
plt.show()

In [ ]:
print("Purity:")
print(purity)

In [ ]:
# accuracy
plt.plot(t,accuracy)
plt.ylim([0,1])
plt.xlabel('Time [unit]')
plt.ylabel("Accuracy")
plt.show()

In [ ]:
print("Accuracy:")
print(accuracy)

In [ ]:
# NMI
plt.plot(t,NMI)
plt.ylim([0,1])
plt.xlabel('Time [unit]')
plt.ylabel("NMI")
plt.show()

In [ ]:
print("NMI:")
print(NMI)